In [0]:
!wget http://www.nactem.ac.uk/tsujii/GENIA/ERtask/Genia4ERtraining.tar.gz
!wget http://www.nactem.ac.uk/tsujii/GENIA/ERtask/Genia4ERtest.tar.gz

--2019-07-12 19:27:22--  http://www.nactem.ac.uk/tsujii/GENIA/ERtask/Genia4ERtraining.tar.gz
Resolving www.nactem.ac.uk (www.nactem.ac.uk)... 130.88.113.155
Connecting to www.nactem.ac.uk (www.nactem.ac.uk)|130.88.113.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2308482 (2.2M) [application/x-gzip]
Saving to: ‘Genia4ERtraining.tar.gz’

Genia4ERtraining.ta 100%[===================>]   2.20M   387KB/s    in 7.4s    

2019-07-12 19:27:36 (305 KB/s) - ‘Genia4ERtraining.tar.gz’ saved [2308482/2308482]

--2019-07-12 19:27:37--  http://www.nactem.ac.uk/tsujii/GENIA/ERtask/Genia4ERtest.tar.gz
Resolving www.nactem.ac.uk (www.nactem.ac.uk)... 130.88.113.155
Connecting to www.nactem.ac.uk (www.nactem.ac.uk)|130.88.113.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862590 (842K) [application/x-gzip]
Saving to: ‘Genia4ERtest.tar.gz’

Genia4ERtest.tar.gz 100%[===================>] 842.37K   184KB/s    in 4.6s    

2019-07-12 19:27:42 (184 

In [0]:
!tar -xf Genia4ERtraining.tar.gz
!tar -xf Genia4ERtest.tar.gz

In [0]:
!pip install flair

     |████████████████████████████████| 143kB 45.6MB/s 
     |████████████████████████████████| 133kB 46.6MB/s 
     |████████████████████████████████| 655kB 43.4MB/s 
     |████████████████████████████████| 798kB 39.6MB/s 
     |████████████████████████████████| 1.0MB 45.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
Successfully built sqlitedict regex mpld3 segtok


In [0]:
from flair.datasets import ColumnCorpus
from flair.data import Corpus


# 1. Get corpus
corpus: Corpus = ColumnCorpus(data_folder='./',
                      column_format={0: 'text', 
                                     1: 'ner'},
                      train_file='Genia4ERtask1.iob2', 
                      test_file='Genia4EReval1.iob2').downsample(0.5)
print(corpus)
for tag in corpus.make_tag_dictionary('ner').get_items():
  print(tag)

2019-07-12 22:48:41,704 Reading data from .
2019-07-12 22:48:41,705 Train: Genia4ERtask1.iob2
2019-07-12 22:48:41,706 Dev: None
2019-07-12 22:48:41,708 Test: Genia4EReval1.iob2
Corpus: 8346 train + 928 dev + 1928 test sentences
<unk>
O
B-protein
I-protein
B-cell_type
I-cell_type
B-DNA
I-DNA
B-cell_line
I-cell_line
B-RNA
I-RNA
<START>
<STOP>


In [0]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings
from typing import List

import matplotlib
%matplotlib inline

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # WordEmbeddings('en-glove'),

    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('pubmed-forward'),
    # FlairEmbeddings('pubmed-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=128,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=10)

# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')

[b'<unk>', b'O', b'B-protein', b'I-protein', b'B-cell_type', b'I-cell_type', b'B-DNA', b'I-DNA', b'B-cell_line', b'I-cell_line', b'B-RNA', b'I-RNA', b'<START>', b'<STOP>']
2019-07-12 23:32:26,779 ----------------------------------------------------------------------------------------------------
2019-07-12 23:32:26,780 Evaluation method: MICRO_F1_SCORE
2019-07-12 23:32:27,918 ----------------------------------------------------------------------------------------------------
2019-07-12 23:32:31,524 epoch 1 - iter 0/131 - loss 97.34165192
2019-07-12 23:32:55,727 epoch 1 - iter 13/131 - loss 39.95286696
2019-07-12 23:33:19,503 epoch 1 - iter 26/131 - loss 33.20195000
2019-07-12 23:33:44,180 epoch 1 - iter 39/131 - loss 29.32883868
2019-07-12 23:34:08,267 epoch 1 - iter 52/131 - loss 26.64831755
2019-07-12 23:34:32,464 epoch 1 - iter 65/131 - loss 24.62835727
2019-07-12 23:34:56,284 epoch 1 - iter 78/131 - loss 22.92910739
2019-07-12 23:35:25,971 epoch 1 - iter 91/131 - loss 21.51573443
2